<a href="https://colab.research.google.com/github/VincentHenric/Deep_learning_project/blob/master/Golois_mod_Corentin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)

2.0.0


In [0]:
#if pybind11 is not installed
#pip install pybind11

     |████████████████████████████████| 153kB 8.7MB/s 


In [0]:
#Connect to our own Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Go on the DL project folder
%cd "/content/gdrive/My Drive/DeepLearningProject/"
!ls

/content/gdrive/My Drive/DeepLearningProject
Board.h        games.data			       policy.npy
compileMAC.sh  games.txt			       __pycache__
compile.sh     golois.cpp			       README
convert        golois.cpython-37m-x86_64-linux-gnu.so  Rzone.h
convert.cpp    golois.py			       save.py
DL_Proj.ipynb  input_data.npy			       value.npy
end.npy        ls.sh
Game.h	       models


In [0]:
#Run golois.py
#!python golois.py

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from datetime import datetime
import os
#import golois

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, MaxPooling2D, Conv2D, BatchNormalization, Activation
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2


planes = 8
moves = 361
dynamicBatch = False
if dynamicBatch:
    N = 100000
    input_data = np.random.randint(2, size=(N, 19, 19, planes))
    input_data = input_data.astype ('float32')
    
    policy = np.random.randint(moves, size=(N,))
    policy = keras.utils.to_categorical (policy)
    
    value = np.random.randint(2, size=(N,))
    value = value.astype ('float32')
    
    end = np.random.randint(2, size=(N, 19, 19, 2))
    end = end.astype ('float32')

    golois.getBatch (input_data, policy, value, end)
else:
    input_data = np.load ('input_data.npy')
    policy = np.load ('policy.npy')
    value = np.load ('value.npy')
    end = np.load ('end.npy')

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(42, 3, padding='same')(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
ident = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.add([ident,x])
ident2 = x
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.Conv2D(128, 3, padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = layers.add([ident2,x])

policy_head = layers.Conv2D(2, 1, padding='same')(x)
policy_head = BatchNormalization()(policy_head)
policy_head = Activation('relu')(policy_head)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(moves, activation='softmax', name='policy')(policy_head)

value_head = layers.Conv2D(1, 1, padding='same')(x)
value_head = BatchNormalization()(value_head)
value_head = Activation('relu')(value_head)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(256, activation='relu')(value_head)
value_head = layers.Dense(1, activation='tanh', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])
model.summary()

#Name of the model, to save it later
now = datetime.now()
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'dl_proj_model_' + date_time + '.h5'



Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 8)]  0                                            
__________________________________________________________________________________________________
conv2d_259 (Conv2D)             (None, 19, 19, 42)   3066        board[0][0]                      
__________________________________________________________________________________________________
batch_normalization_249 (BatchN (None, 19, 19, 42)   168         conv2d_259[0][0]                 
__________________________________________________________________________________________________
activation_245 (Activation)     (None, 19, 19, 42)   0           batch_normalization_249[0][0]    
___________________________________________________________________________________________

In [0]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir, model_name)
#Save only the best model
#Problem : multiple output // j'ai pas encore vu comment checker les deux outputs
model_save = ModelCheckpoint(model_path, monitor='val_policy_loss', verbose=1, save_best_only=True, mode='min')
#Avoid wasting time
earlyStopping = EarlyStopping(monitor="val_policy_loss",patience=15)
callbacks_list = [model_save, earlyStopping]

model.compile(optimizer='Adam',
              loss={'value': 'mse', 'policy': 'categorical_crossentropy'})

model.fit(input_data, {'policy': policy, 'value': value},
           epochs=40, batch_size=128, validation_split=0.2, callbacks=callbacks_list)

print("Saved model : " + model_path)


Train on 80000 samples, validate on 20000 samples
Epoch 1/40
79744/80000 [============================>.] - ETA: 0s - loss: 5.4353 - policy_loss: 5.1803 - value_loss: 0.2550
Epoch 00001: val_policy_loss improved from inf to 4.65521, saving model to /content/gdrive/My Drive/DeepLearningProject/models/dl_proj_model_12-04-2019_22-28-48.h5
80000/80000 [==============================] - 20s 254us/sample - loss: 5.4333 - policy_loss: 5.1783 - value_loss: 0.2550 - val_loss: 4.8975 - val_policy_loss: 4.6552 - val_value_loss: 0.2410
Epoch 2/40
79744/80000 [============================>.] - ETA: 0s - loss: 4.5950 - policy_loss: 4.3536 - value_loss: 0.2414
Epoch 00002: val_policy_loss improved from 4.65521 to 4.25313, saving model to /content/gdrive/My Drive/DeepLearningProject/models/dl_proj_model_12-04-2019_22-28-48.h5
80000/80000 [==============================] - 19s 243us/sample - loss: 4.5943 - policy_loss: 4.3529 - value_loss: 0.2414 - val_loss: 4.5008 - val_policy_loss: 4.2531 - val_value

In [0]:
#Run and evaluate  the last
import glob
from tensorflow.keras.models import load_model

#Take the latest_file, i.e. the latest model
save_dir = os.path.join(os.getcwd(), 'models')
save_dir = save_dir+'/*'
list_of_files = glob.glob(save_dir) 
latest_file = max(list_of_files, key=os.path.getctime)
model_name = latest_file

model_path = model_name
model = load_model(model_path)

score = model.evaluate(x_test,y_test,batch_size=64,verbose = 0)
print ('Validation accuracy', model.metrics_names, score)

NameError: ignored